##Ce notebook permet de récupérer les données de consommation électrique sur le site RTE et les pré-traiter pour obtenir un df de relevé de consommation pour chaque 30 mn  par région et nationale des années 2021 et 2022 


In [ ]:
#acceder à mon drive pour stocker les données
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#!pip install -U holidays

In [ ]:
#!pip install --upgrade pandas


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.2 MB 4.2 MB/s 
  Attempting uninstall: pandas
    Found existing installation: pandas 1.3.5
    Uninstalling pandas-1.3.5:
      Successfully uninstalled pandas-1.3.5


In [ ]:
!pip install --upgrade xlrd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 96 kB 3.1 MB/s 
  Attempting uninstall: xlrd
    Found existing installation: xlrd 1.2.0
    Uninstalling xlrd-1.2.0:
      Successfully uninstalled xlrd-1.2.0


In [ ]:
#chargement des librairies
import pandas as pd
import numpy as np
import os
import subprocess
import calendar
from datetime import datetime
import datetime
import glob
import holidays


In [ ]:
# chargement data sur le site RTE de l'année  en cours 2022
#! wget -O data_elect_encours.zip "https://eco2mix.rte-france.com/download/eco2mix/eCO2mix_RTE_En-cours-TR.zip"
#!unzip -qq data_elect_encours.zip

In [ ]:
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pd.set_option('display.max_columns', 45)

## preparation de données pour la prévision de la consommation électrique en France

##récupération  de données de consommation  sur le site RTE

In [ ]:
url_region={"Auvergne":"https://eco2mix.rte-france.com/download/eco2mix/eCO2mix_RTE_Auvergne-Rh%C3%B4ne-Alpes_",
            "Bourgogne":"https://eco2mix.rte-france.com/download/eco2mix/eCO2mix_RTE_Bourgogne-Franche-Comt%C3%A9_",
            "Bretagne":"https://eco2mix.rte-france.com/download/eco2mix/eCO2mix_RTE_Bretagne_",
            "Centre_loire":"https://eco2mix.rte-france.com/download/eco2mix/eCO2mix_RTE_Centre-Val-de-Loire_",
            "Grand_est":"https://eco2mix.rte-france.com/download/eco2mix/eCO2mix_RTE_Grand-Est_",
            "Haut_france":"https://eco2mix.rte-france.com/download/eco2mix/eCO2mix_RTE_Hauts-de-France_",
            "Ile_france":"https://eco2mix.rte-france.com/download/eco2mix/eCO2mix_RTE_Ile-de-France_",
            "Normandie":"https://eco2mix.rte-france.com/download/eco2mix/eCO2mix_RTE_Normandie_",
            "Aquitaine":"https://eco2mix.rte-france.com/download/eco2mix/eCO2mix_RTE_Nouvelle-Aquitaine_",
            "Occitanie":"https://eco2mix.rte-france.com/download/eco2mix/eCO2mix_RTE_Occitanie_",
            "PACA":"https://eco2mix.rte-france.com/download/eco2mix/eCO2mix_RTE_PACA_",
            "Pays_loire":"https://eco2mix.rte-france.com/download/eco2mix/eCO2mix_RTE_Pays-de-la-Loire_"
             }

In [ ]:

# récupération  de données des année 2013 et 2020
url="/content/drive/MyDrive/MALIA/FORCASTING/projet_forecasting/data_consommation_regionale"
os.chdir(url)
"""
for i in range(2013,2021):
  for region, url in url_region.items():
    subprocess.run(["wget", "-O", "data_"+region+".zip", url+"Annuel-Definitif_"+str(i)+".zip"])
    subprocess.run(["unzip", "-qq", "data_"+region+".zip"])
"""
# récupération  de données des année 2021 et 2022
URL_cours_consolite=["En-cours-Consolide.zip","En-cours-TR.zip"]
for url_cours in URL_cours_consolite:
  for region, url in url_region.items():
    subprocess.run(["wget", "-O", "data_"+region+".zip", url+url_cours])
    subprocess.run(["unzip", "-qq", "data_"+region+".zip"])

In [ ]:
#récupération du chemin de chaque fihier
fold_file="/content/drive/MyDrive/MALIA/FORCASTING/projet_forecasting/data_consommation_regionale"
os.chdir(fold_file)
files=glob.glob(os.path.join(fold_file,"*.xls"))
print(len(files))

26


##" fonction pour charger tous les fichiers csv de  données stockés dans le drive

In [ ]:
def load_file(folder_path,csv_files,delimiter=",",encoding='utf-8'):
      # Liste des fichiers .xls dans le dossier


      # Initialisation d'un dataframe vide
      df = pd.DataFrame()

      # Pour chaque fichier .xls dans le dossier, téléchargement et concaténation dans le dataframe
      for i,fichier  in enumerate(csv_files):
          file_path = os.path.join(folder_path, fichier)
          print("-"*100)
          print(fichier)
          print(i)
          #xlsx = pd.ExcelFile(file_path,engine="")
          #df_temp= pd.read_excel(xlsx, "Sheet1")
          
          #df_temp = pd.read_excel(file_path,engine="xlrd")
          df_temp = pd.read_csv(file_path, low_memory=False,delimiter=delimiter,encoding=encoding)
          print("-"*100)
          df = pd.concat([df, df_temp], ignore_index=True)
      return df


# chargement de données par région 2021 et 2022

In [ ]:
# Chemin d'accès au dossier contenant les fichiers .xls
folder_path="/content/drive/MyDrive/MALIA/FORCASTING/projet_forecasting/data_consommation_regionale"
os.chdir(folder_path)
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
len(csv_files)
df1=load_file(folder_path,csv_files,encoding='latin-1',delimiter=";")

----------------------------------------------------------------------------------------------------
eCO2mix_RTE_Auvergne-Rh ne-Alpes_En-cours-Consolide.csv
0
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
eCO2mix_RTE_Bourgogne-Franche-Comt _En-cours-Consolide.xls.csv
1
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
eCO2mix_RTE_Bretagne_En-cours-Consolide.xls.csv
2
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
eCO2mix_RTE_Centre-Val-de-Loire_En-cours-Consolide.xls.csv
3
-----------------------------------------------------------

In [ ]:
df1.head()

,Périmètre,Nature,Date,Heures,Consommation,Thermique,Nucléaire,Eolien,Solaire,Hydraulique,Pompage,Bioénergies,Stockage batterie,Déstockage batterie,Eolien terrestre,Eolien offshore,Ech. physiques,TCO Thermique (%),TCH Thermique (%),TCO Nucléaire (%),TCH Nucléaire (%),TCO Eolien (%),...,Flux physiques de PACA vers Luxembourg,Flux physiques de PACA vers Royaume-Uni,Flux physiques de PACA vers Suisse,Flux physiques d'Auvergne-Rhône-Alpes vers Pays-de-la-Loire,Flux physiques d'Ile-de-France vers Pays-de-la-Loire,Flux physiques d'Occitanie vers Pays-de-la-Loire,Flux physiques de Pays-de-la-Loire vers Pays-de-la-Loire,Flux physiques de Pays-de-la-Loire vers Pays-de-la-Loire.1,Flux physiques Allemagne vers Pays-de-la-Loire,Flux physiques Belgique vers Pays-de-la-Loire,Flux physiques Espagne vers Pays-de-la-Loire,Flux physiques Italie vers Pays-de-la-Loire,Flux physiques Luxembourg vers Pays-de-la-Loire,Flux physiques Royaume-Uni vers Pays-de-la-Loire,Flux physiques Suisse vers Pays-de-la-Loire,Flux physiques de Pays-de-la-Loire vers Allemagne,Flux physiques de Pays-de-la-Loire vers Belgique,Flux physiques de Pays-de-la-Loire vers Espagne,Flux physiques de Pays-de-la-Loire vers Italie,Flux physiques de Pays-de-la-Loire vers Luxembourg,Flux physiques de Pays-de-la-Loire vers Royaume-Uni,Flux physiques de Pays-de-la-Loire vers Suisse
0,Auvergne-Rhône-Alpes,Données consolidées,01/01/2021,00:00,8576.0,157.0,10710.0,62.0,0.0,3076.0,-167.0,137.0,-5400,1.83,14.58,124.88,78.92,0.72,11.09,0.0,0.0,35.87,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Auvergne-Rhône-Alpes,Données consolidées,01/01/2021,00:15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Auvergne-Rhône-Alpes,Données consolidées,01/01/2021,00:30,8514.0,172.0,10622.0,82.0,0.0,3010.0,-164.0,132.0,-5340,2.02,15.97,124.76,78.28,0.96,14.67,0.0,0.0,35.35,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Auvergne-Rhône-Alpes,Données consolidées,01/01/2021,00:45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Auvergne-Rhône-Alpes,Données consolidées,01/01/2021,01:00,8273.0,175.0,10652.0,78.0,0.0,2699.0,-163.0,132.0,-5299,2.12,16.25,128.76,78.5,0.94,13.95,0.0,0.0,32.62,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
pd.set_option('display.max_columns', 150)

In [ ]:
#sélection des 13 premières colonnes
col_keep=df1.columns[:13]
"""
col_keep=["Périmètre", "Nature", "Date", "Heures", "Consommation", "Thermique",
       "Nucléaire", "Eolien", "Solaire", "Hydraulique", "Pompage",
       "Bioénergies", "Ech. physiques"]
"""
col_keep

Index(['Périmètre', 'Nature', 'Date', 'Heures', 'Consommation', 'Thermique',
       'Nucléaire', 'Eolien', 'Solaire', 'Hydraulique', 'Pompage',
       'Bioénergies', 'Stockage batterie'],
      dtype='object')

In [ ]:
#filtre du df des variables sélectionnées
df1=df1[col_keep]
df =df1.copy()
df.shape

(843288, 13)

In [ ]:
#nombre des valeurs manquantes de la colonne "consommation"
df["Consommation"].isna().sum()

297708

In [ ]:
# fonction de traitement de données données consommation électrique

def traitementDataConsoRTE(df):
      #df["datetime"]=pd.to_datetime(df["Date"], format='%d/%m/%Y')

        #on cancatene les colonnes date et heure pour avoir un timestamp
      
      df=df[["Périmètre","Date","Heures","Consommation"]]
      # supprime les lignes qui ont pour valeur ND et NAN de la variable con
      df = df[~df['Consommation'].isin(['ND',np.NAN])]
      df = df.reset_index(drop=True)
      df["datetime"]=pd.to_datetime(df["Date"], format='%d/%m/%Y')
      df["year"] = df["datetime"].dt.year
      df['heure'] = pd.to_datetime(df['Heures'], format='%H:%M').dt.time

      # création varaible timestamp et ordonner le df par cette variable
      df['timestamp'] = pd.to_datetime(df.Date +' '+df.heure.astype(str))
      df = df.sort_values(by='timestamp', ascending=True)
      
      #suppression toutes les lignes dont toutes les valeurs sont des NAN
      df = df.dropna(how='all')
      df = df.reset_index(drop=True)
      return df

In [ ]:
# application de la fonction de pré-traitement df  de conso electrique 
d=df.copy()
df_conso=traitementDataConsoRTE(d)

In [ ]:
df_conso

,Périmètre,Date,Heures,Consommation,datetime,year,heure,timestamp
0,Auvergne-Rhône-Alpes,01/01/2021,00:00,8576.0,2021-01-01,2021,00:00:00,2021-01-01 00:00:00
1,Pays-de-la-Loire,01/01/2021,00:00,4230.0,2021-01-01,2021,00:00:00,2021-01-01 00:00:00
2,Centre-Val de Loire,01/01/2021,00:00,2900.0,2021-01-01,2021,00:00:00,2021-01-01 00:00:00
3,Grand-Est,01/01/2021,00:00,5321.0,2021-01-01,2021,00:00:00,2021-01-01 00:00:00
4,Hauts-de-France,01/01/2021,00:00,6340.0,2021-01-01,2021,00:00:00,2021-01-01 00:00:00
...,...,...,...,...,...,...,...,...
543888,PACA,02/01/2023,14:00,4592,2023-01-02,2023,14:00:00,2023-02-01 14:00:00
543889,Hauts-de-France,02/01/2023,14:00,6686,2023-01-02,2023,14:00:00,2023-02-01 14:00:00
543890,Auvergne-Rhône-Alpes,02/01/2023,14:00,7469,2023-01-02,2023,14:00:00,2023-02-01 14:00:00
543891,Grand-Est,02/01/2023,14:00,4878,2023-01-02,2023,14:00:00,2023-02-01 14:00:00


In [ ]:
df_conso.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 543893 entries, 0 to 543892
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   Périmètre     543893 non-null  object        
 1   Date          543893 non-null  object        
 2   Heures        543893 non-null  object        
 3   Consommation  543893 non-null  object        
 4   datetime      543893 non-null  datetime64[ns]
 5   year          543893 non-null  int64         
 6   heure         543893 non-null  object        
 7   timestamp     543893 non-null  datetime64[ns]
dtypes: datetime64[ns](2), int64(1), object(5)
memory usage: 33.2+ MB


In [ ]:
#sélection des variables
df_final_conso=df_conso[["Périmètre", "datetime", "year","timestamp","heure","Consommation"]]

In [ ]:
#filtre du df 
df_final_conso=df_conso[["Périmètre", "datetime", "year","timestamp","heure","Consommation"]]

# renommer la variable "périmètre" en "region"
df_final_conso=df_final_conso.rename(columns={"Périmètre":"region"})
# renommer la variable "timestamp" en "DateHeure"
df_final_conso=df_final_conso.rename(columns={"timestamp":"DateHeure"})
df_final_conso['DateHeure'] = pd.to_datetime(df_final_conso['DateHeure'],utc=True)

df_final_conso["Consommation"]=df_final_conso["Consommation"].astype(int)
# renommer la variable "Consommation" en "conso"
df_final_conso=df_final_conso.rename(columns={"Consommation":"conso"})

# Filtrer le DataFrame en ne conservant que les lignes où year est égal à 2021 ou 2022
df_conso_filter = df_final_conso.loc[(df_final_conso["year"]  == 2021) | (df_final_conso["year"] == 2022)]

# Supprimer les lignes qui ont une datetime en minutes (15 ou 45)
df_conso_region=df_conso_filter.copy()
df_conso_region = df_conso_region[~df_conso_region['DateHeure'].dt.minute.isin([15,45])]
print(df_conso_region.shape)

(419712, 6)


In [ ]:
# enregistrement du df final de consommation électrique parr région des années 2021 et 2022
directory="/content/drive/MyDrive/MALIA/FORCASTING/projet_forecasting/data_RTE_METEO"
os.chdir(directory)
df_conso_region.to_csv("df_consommation.csv",index=False)

In [ ]:
df_final_conso["region"].unique()

array(['Auvergne-Rhône-Alpes', 'Pays-de-la-Loire', 'Centre-Val de Loire',
       'Grand-Est', 'Hauts-de-France', 'Bretagne', 'Ile-de-France',
       'Nouvelle-Aquitaine', 'Bourgogne-Franche-Comté', 'Occitanie',
       'PACA', 'Normandie'], dtype=object)

In [ ]:
df_final_conso

,region,datetime,year,timestamp,heure,Consommation
0,Auvergne-Rhône-Alpes,2021-01-01,2021,2021-01-01 00:00:00,00:00:00,8576
1,Pays-de-la-Loire,2021-01-01,2021,2021-01-01 00:00:00,00:00:00,4230
2,Centre-Val de Loire,2021-01-01,2021,2021-01-01 00:00:00,00:00:00,2900
3,Grand-Est,2021-01-01,2021,2021-01-01 00:00:00,00:00:00,5321
4,Hauts-de-France,2021-01-01,2021,2021-01-01 00:00:00,00:00:00,6340
...,...,...,...,...,...,...
543888,PACA,2023-01-02,2023,2023-02-01 14:00:00,14:00:00,4592
543889,Hauts-de-France,2023-01-02,2023,2023-02-01 14:00:00,14:00:00,6686
543890,Auvergne-Rhône-Alpes,2023-01-02,2023,2023-02-01 14:00:00,14:00:00,7469
543891,Grand-Est,2023-01-02,2023,2023-02-01 14:00:00,14:00:00,4878


###données conso electrique  nationnales

In [ ]:
# Chemin d'accès au dossier contenant les fichiers .xls
folder_path="/content/drive/MyDrive/MALIA/FORCASTING/projet_forecasting/conso_nationale"
os.chdir(folder_path)
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
len(csv_files)
df_nat=load_file(folder_path,csv_files,encoding='latin-1',delimiter=";")

----------------------------------------------------------------------------------------------------
eCO2mix_RTE_En-cours-Consolide.xls.csv
0
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
eCO2mix_RTE_En-cours-TR.xls.csv
1
----------------------------------------------------------------------------------------------------


In [ ]:
df_nat.shape

(71138, 40)

In [ ]:
col_keep=df_nat.columns[:13]
df_nat=df_nat[col_keep]
dfN=df_nat.copy()

In [ ]:
dN=dfN.copy()
dfN_conso=traitementDataConsoRTE(dN)

In [ ]:
dfN_conso

,Périmètre,Date,Heures,Consommation,datetime,year,heure,timestamp
0,France,01/01/2021,00:00,67010.0,2021-01-01,2021,00:00:00,2021-01-01 00:00:00
1,France,01/01/2021,00:30,67071.0,2021-01-01,2021,00:30:00,2021-01-01 00:30:00
2,France,01/01/2021,01:00,65052.0,2021-01-01,2021,01:00:00,2021-01-01 01:00:00
3,France,01/01/2021,01:30,64918.0,2021-01-01,2021,01:30:00,2021-01-01 01:30:00
4,France,01/01/2021,02:00,64376.0,2021-01-01,2021,02:00:00,2021-01-01 02:00:00
...,...,...,...,...,...,...,...,...
46243,France,10/01/2023,16:45,60251.0,2023-01-10,2023,16:45:00,2023-10-01 16:45:00
46244,France,10/01/2023,17:00,60273.0,2023-01-10,2023,17:00:00,2023-10-01 17:00:00
46245,France,10/01/2023,17:15,60711.0,2023-01-10,2023,17:15:00,2023-10-01 17:15:00
46246,France,10/01/2023,17:30,61739.0,2023-01-10,2023,17:30:00,2023-10-01 17:30:00


In [ ]:
df_final_conso=dfN_conso[["Périmètre", "datetime", "year","timestamp","heure","Consommation"]]
df_final_conso=df_final_conso.rename(columns={"Périmètre":"region"})
df_final_conso["Consommation"]=df_final_conso["Consommation"].astype(int)

# Filtrer le DataFrame en ne conservant que les lignes où year est égal à 2021 ou 2022
df_conso_filter = df_final_conso.loc[(df_final_conso["year"]  == 2021) | (df_final_conso["year"] == 2022)]

# Supprimer les lignes qui ont une datetime en minutes (15 ou 45)
df_conso_filter_clean=df_conso_filter.copy()
df_conso_nat= df_conso_filter_clean[~df_conso_filter_clean['timestamp'].dt.minute.isin([15,45])]

# renommer la variable "timestamp" en "DateHeure"
df_conso_nat=df_conso_nat.rename(columns={"timestamp":"DateHeure"})
df_conso_nat['DateHeure'] = pd.to_datetime(df_conso_nat['DateHeure'],utc=True)

df_conso_nat["Consommation"]=df_conso_nat["Consommation"].astype(int)
# renommer la variable "Consommation" en "conso"
df_conso_nat=df_conso_nat.rename(columns={"Consommation":"conso"})

print(df_conso_nat.shape)

(35040, 6)


In [ ]:
df_conso_nat

,region,datetime,year,DateHeure,heure,conso
0,France,2021-01-01,2021,2021-01-01 00:00:00+00:00,00:00:00,67010
1,France,2021-01-01,2021,2021-01-01 00:30:00+00:00,00:30:00,67071
2,France,2021-01-01,2021,2021-01-01 01:00:00+00:00,01:00:00,65052
3,France,2021-01-01,2021,2021-01-01 01:30:00+00:00,01:30:00,64918
4,France,2021-01-01,2021,2021-01-01 02:00:00+00:00,02:00:00,64376
...,...,...,...,...,...,...
45302,France,2022-12-31,2022,2022-12-31 21:30:00+00:00,21:30:00,43971
45304,France,2022-12-31,2022,2022-12-31 22:00:00+00:00,22:00:00,43506
45306,France,2022-12-31,2022,2022-12-31 22:30:00+00:00,22:30:00,44566
45308,France,2022-12-31,2022,2022-12-31 23:00:00+00:00,23:00:00,46232


In [ ]:
# enregistrement des données nationnales
directory="/content/drive/MyDrive/MALIA/FORCASTING/projet_forecasting/data_RTE_METEO"
os.chdir(directory)
df_conso_nat.to_csv("df_conso_nationale.csv",index=False)

In [ ]:
df_conso_nat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35040 entries, 0 to 45310
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   region        35040 non-null  object        
 1   datetime      35040 non-null  datetime64[ns]
 2   year          35040 non-null  int64         
 3   timestamp     35040 non-null  datetime64[ns]
 4   heure         35040 non-null  object        
 5   Consommation  35040 non-null  int64         
dtypes: datetime64[ns](2), int64(2), object(2)
memory usage: 1.9+ MB


In [ ]:
df_conso_nat.isna().sum()

region          0
datetime        0
year            0
timestamp       0
heure           0
Consommation    0
dtype: int64

#Dans cette partie on va charger les données météo depuis le site  météo France et les prétraiter pour obtenir un df final avec un intervalle de 30 min de  rélévé météo.effectuer les prétraiements


## récupération de données météo sur le site[data météo](https://public.opendatasoft.com/explore/dataset/donnees-synop-essentielles-omm/table/?sort=date)
et

In [ ]:
#stockage de données sur mon drive
path_url="/content/drive/MyDrive/MALIA/FORCASTING/projet_forecasting/data_meteo"
os.chdir(path_url)
##extraction de données météo sur le site
##url data météo 2021/2022: https://data.opendatasoft.com/explore/dataset/donnees-synop-essentielles-omm%40public/export/?sort=date&q.timerange.date=date:%5B2020-12-31T23:00:00Z+TO+2022-12-31T22:59:59Z%5D
!wget -O data_meteo.csv https://data.opendatasoft.com/explore/dataset/donnees-synop-essentielles-omm@public/download/?format=csv&q=date:%5B2020-12-31T23:00:00Z+TO+2022-12-31T22:59:59Z%5D&timezone=Europe/Berlin&lang=fr&use_labels_for_header=true&csv_separator=%3B
#!unzip -qq data_meteo.zip 

--2023-02-05 09:26:23--  https://data.opendatasoft.com/explore/dataset/donnees-synop-essentielles-omm@public/download/?format=csv
Resolving data.opendatasoft.com (data.opendatasoft.com)... 34.249.199.226, 34.248.20.69
Connecting to data.opendatasoft.com (data.opendatasoft.com)|34.249.199.226|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/csv]
Saving to: ‘data_meteo.csv’

data_meteo.csv          [            <=>     ]  20.95M   815KB/s               

KeyboardInterrupt: ignored

In [ ]:
# chargement de données météo
import time
# temps de chargment
start_time = time.perf_counter()

df=pd.read_csv("data_meteoV4.csv",delimiter=",")#,nrows=20_000
end_time = time.perf_counter()
print(end_time-start_time," secondes")
df.shape

33.130769482999995  secondes


(2193942, 82)

In [ ]:
df.head(3)

,numer_sta,date,pmer,tend,cod_tend,dd,ff,t,td,u,vv,ww,w1,w2,n,nbas,hbas,cl,cm,ch,pres,niv_bar,geop,tend24,tn12,tn24,tx12,tx24,tminsol,sw,tw,raf10,rafper,per,etat_sol,ht_neige,ssfrai,perssfrai,rr1,rr3,rr6,rr12,rr24,phenspe1,phenspe2,phenspe3,phenspe4,nnuage1,ctype1,hnuage1,nnuage2,ctype2,hnuage2,nnuage3,ctype3,hnuage3,nnuage4,ctype4,hnuage4,coordonnees,nom,type_de_tendance_barometrique,temps_passe_1,temps_present,tc,tn12c,tn24c,tx12c,tx24c,tminsolc,latitude,longitude,altitude,libgeo,codegeo,nom_epci,code_epci,nom_dept,code_dep,nom_reg,code_reg,mois_de_l_annee
0,7558,2010-01-05T09:00:00+00:00,100280.0,-50.0,5.0,260.0,1.5,275.75,275.75,100.0,4000.0,50.0,5.0,5.0,100.0,8.0,250.0,37.0,61.0,60.0,91870.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.1,-10.0,NaN,NaN,0.0,-30.0,0.0,0.4,NaN,NaN,NaN,710.0,1102.0,3100.0,NaN,6.0,7.0,180.0,7.0,6.0,480.0,NaN,NaN,NaN,NaN,NaN,NaN,"44.1185,3.0195",MILLAU,"En baisse, puis en hausse, la pression atmosph...",Bruine,"Bruine, sans congélation, intermittente, faibl...",2.6,NaN,NaN,NaN,NaN,NaN,44.118500,3.019500,712,Millau,12145,CC de Millau Grands Causses,241200567.0,Aveyron,12,Occitanie,76.0,1
1,61976,2010-01-05T09:00:00+00:00,100990.0,NaN,NaN,NaN,NaN,305.45,299.05,69.0,50000.0,2.0,1.0,1.0,60.0,1.0,800.0,32.0,20.0,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,2439.0,NaN,NaN,NaN,1.0,8.0,600.0,5.0,0.0,7500.0,NaN,NaN,NaN,NaN,NaN,NaN,"-15.887667,54.520667",TROMELIN,NaN,Nuages couvrant plus de la moitié du ciel pend...,État du ciel inchangé dans l’ensemble,32.3,NaN,NaN,NaN,NaN,NaN,-15.887667,54.520667,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,7027,2010-01-05T12:00:00+00:00,100720.0,-190.0,8.0,200.0,3.6,273.65,271.75,87.0,12000.0,2.0,1.0,1.0,75.0,6.0,800.0,35.0,20.0,10.0,99880.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.1,-10.0,NaN,NaN,0.0,-60.0,0.0,0.0,0.0,NaN,NaN,710.0,1105.0,3100.0,NaN,3.0,6.0,810.0,6.0,6.0,1200.0,NaN,NaN,NaN,NaN,NaN,NaN,"49.18,-0.456167",CAEN-CARPIQUET,"Stationnaire ou en hausse, puis en baisse, ou ...",Nuages couvrant plus de la moitié du ciel pend...,État du ciel inchangé dans l’ensemble,0.5,NaN,NaN,NaN,NaN,NaN,49.180000,-0.456167,67,Carpiquet,14137,CU Caen la Mer,200065597.0,Calvados,14,Normandie,28.0,1


In [ ]:
#je selectionne quelques varaibles importantes 
temp=df.iloc[:,:8]
#chargement données station-region stocker dans le drive
df_station=pd.read_csv("region_station_meteo.csv",delimiter=";" , encoding="latin-1")
region_dict = df_station.set_index('ID_station')['Région'].to_dict()

#ajout varigee "region" dans df "temp  afin de faire corresponde 
#chaque stationà sa region à partir la colonnes "numer_sta" et "ID_station
temp['region'] = temp['numer_sta'].map(region_dict)

#convertir les variables en datetime
temp['DateHeure'] = pd.to_datetime(df['date'])
temp["year"] = temp["DateHeure"].dt.year
temp=temp.sort_values(by=["year","DateHeure"])
temp = temp.reset_index(drop=True)
temp=temp.iloc[:,[0,7,8,9,10]]
# le df météo  final après l'ajout de la variable région
temp.head(4)

,numer_sta,t,region,DateHeure,year
0,7037,273.45,Normandie,2010-01-01 00:00:00+00:00,2010
1,78897,298.75,Mayotte,2010-01-01 00:00:00+00:00,2010
2,7661,285.35,PACA,2010-01-01 00:00:00+00:00,2010
3,61968,300.95,Mayotte,2010-01-01 00:00:00+00:00,2010


In [ ]:
#liste des régions
regions=temp['region'].unique().tolist()
print("total région:",len(regions))
regions

total région: 14


['Normandie',
 'Mayotte',
 'PACA',
 'Pays-de-la-Loire',
 'CORSE',
 'Nouvelle-Aquitaine',
 'Bretagne',
 'Centre-Val de Loire',
 'Grand-Est',
 'Auvergne-Rhône-Alpes',
 'Hauts-de-France',
 'Occitanie',
 'Bourgogne-Franche-Comté',
 'Ile-de-France']

In [ ]:
#création d'un df datetime intervallle de 3h
def date_to_range3H():
      """
      entrée:None
      return: un df datetime de 3h d'intervalle
      """
      datetime = pd.date_range(start="2021-01-01", end="2023-01-01", freq='3H')
      dt = pd.DataFrame(index=datetime)
      dt=dt[:-1]
      dt["DateHeure"]=dt.index
      dt['DateHeure'] = pd.to_datetime(dt['DateHeure'],utc=True)
      dt['year1'] = dt['DateHeure'].dt.year
      dt = dt.reset_index(drop=True)
      return  dt

def agg_func(x):
    """
    x: chaque ligne de mon df
    return: un dict 
    Note: calcule la moyenne  pour chaque région pour chaque 3h
    car on a plusieurs releveé dnas une meme région à 
    chaque heure (plusierus station de rélevé météo)
    """
    return pd.DataFrame.from_dict({
        't': x['t'].mean(),
        #'region': x['region'].unique() #  'region': ''.join(x['region'].unique())
        #'region': ''.join(x['region'].unique())
        'region':','.join(x['region'].astype(str).unique())
        #'year':''.join(str(x['year'].unique()))
    }, orient='index').transpose()

years=[2021,2022]
def df_of_region(df,year=years):
    """
    df: la dat frame temp
    years:lst d'année , pour filtrer notre df
    return: un df 
    """
    df_of_region=pd.DataFrame()
    for region in df['region'].unique():
        for year in years:
            temp=df.loc[(df["region"]==region) & (df["year"]==year)]
            df_date3h=date_to_range3H()
            df_date3h=df_date3h.loc[df_date3h["year1"]==year]
            # concatenantion
            df_merged = pd.merge(df_date3h, temp, on='DateHeure', how='right')
            temp=df_merged.copy()
            #df_of_region = pd.concat([df_of_region, temp])
            # Groupby la colonne datetime et appliquer la fonction
       
            temp= temp.groupby('DateHeure').apply(agg_func).reset_index()
            temp['year'] = temp['DateHeure'].dt.year
            temp=temp[['DateHeure','year', 'region','t']]
            df_of_region = pd.concat([df_of_region, temp])
            
    return df_of_region

In [ ]:
#création de la base de données en appliquant les fonctions d'en haut
years=[2021,2022]
df_meteo=df_of_region(temp,year=years)
df_meteo.shape

(81712, 4)

In [ ]:
#vérification des dimension pour chaque région
for region in regions:
      T=df_meteo.loc[(df_meteo["region"]==region)]
      print(region," : ",T.shape)

Normandie  :  (5838, 4)
Mayotte  :  (5838, 4)
PACA  :  (5838, 4)
Pays-de-la-Loire  :  (5838, 4)
CORSE  :  (5837, 4)
Nouvelle-Aquitaine  :  (5838, 4)
Bretagne  :  (5838, 4)
Centre-Val de Loire  :  (5838, 4)
Grand-Est  :  (5838, 4)
Auvergne-Rhône-Alpes  :  (5838, 4)
Hauts-de-France  :  (5838, 4)
Occitanie  :  (5838, 4)
Bourgogne-Franche-Comté  :  (5822, 4)
Ile-de-France  :  (5835, 4)


In [ ]:
#filtre sur la region Normandie
temp=df_meteo.copy()
temp1=temp.loc[(temp["region"]=="Normandie") & (temp["year"]==2022)]
temp1=temp1[temp1.columns[1:]]
temp1

,year,region,t
0,2022,Normandie,285.125
1,2022,Normandie,284.7
2,2022,Normandie,283.0
3,2022,Normandie,283.325
4,2022,Normandie,286.575
...,...,...,...
2915,2022,Normandie,286.775
2916,2022,Normandie,287.425
2917,2022,Normandie,287.75
2918,2022,Normandie,287.95


In [ ]:
"""
création d'un df datime avec un intervalle de 30mn
# dans "df temp", les températures sont rélevées pour chaque 3h or 
#pour   le fusionner  avec le "df consommation électrique", les données
# doivent etre rélevé dans le meme inetrvalle de temps soit 30mn
"""
def create_date_range(start_date = '2021-01-01',end_date = '2023-01-01'):
      # Créer un index de datetime avec une fréquence de 30 minutes
      datetime_index = pd.date_range(start=start_date, end=end_date, freq='30T')

      # Créer une colonne datetime à partir de l'index
      df = pd.DataFrame(index=datetime_index)

      # Créer un masque booléen pour filtrer les demies-heures (00h30, 3h30, etc.)
      mask =(df.index.hour == 0) & (df.index.minute == 00) |(df.index.hour == 3) & (df.index.minute == 00) | (df.index.hour == 6) & (df.index.minute == 00) | (df.index.hour == 9) & (df.index.minute == 00) | (df.index.hour == 12) & (df.index.minute == 00) | (df.index.hour == 15) & (df.index.minute == 00) | (df.index.hour == 18) & (df.index.minute == 00) | (df.index.hour == 21) & (df.index.minute == 00)

      # Appliquer le masque pour filtrer les dates
      df = df[~mask]

      df["DateHeure"]=df.index
      df['year'] = df['DateHeure'].dt.year
      df = df.reset_index(drop=True)
      return df


In [ ]:
# aplication de la fonction pour créer un df  avec un intervalle de 30mn
df_datetime_manquantes=pd.DataFrame()
datetime_manquantes=create_date_range()
for region in regions:
  temp=datetime_manquantes
  temp["region"]=region
  df_datetime_manquantes = pd.concat([df_datetime_manquantes, temp])
  df_datetime_manquantes["t"]=np.nan

In [ ]:
#visualisation des quelques lignes
df_datetime_manquantes.head(15)

,DateHeure,year,region,t
0,2021-01-01 00:30:00,2021,Normandie,NaN
1,2021-01-01 01:00:00,2021,Normandie,NaN
2,2021-01-01 01:30:00,2021,Normandie,NaN
3,2021-01-01 02:00:00,2021,Normandie,NaN
4,2021-01-01 02:30:00,2021,Normandie,NaN
5,2021-01-01 03:30:00,2021,Normandie,NaN
6,2021-01-01 04:00:00,2021,Normandie,NaN
7,2021-01-01 04:30:00,2021,Normandie,NaN
8,2021-01-01 05:00:00,2021,Normandie,NaN
9,2021-01-01 05:30:00,2021,Normandie,NaN


In [ ]:
#trie du df_datetime  par  region et dateime
df_datetime=df_datetime_manquantes.sort_values(by=["region","DateHeure"])
df_datetime

,DateHeure,year,region,t
0,2021-01-01 00:30:00,2021,Auvergne-Rhône-Alpes,NaN
1,2021-01-01 01:00:00,2021,Auvergne-Rhône-Alpes,NaN
2,2021-01-01 01:30:00,2021,Auvergne-Rhône-Alpes,NaN
3,2021-01-01 02:00:00,2021,Auvergne-Rhône-Alpes,NaN
4,2021-01-01 02:30:00,2021,Auvergne-Rhône-Alpes,NaN
...,...,...,...,...
29195,2022-12-31 21:30:00,2022,Pays-de-la-Loire,NaN
29196,2022-12-31 22:00:00,2022,Pays-de-la-Loire,NaN
29197,2022-12-31 22:30:00,2022,Pays-de-la-Loire,NaN
29198,2022-12-31 23:00:00,2022,Pays-de-la-Loire,NaN


In [ ]:
#concatenation df_datetime manquante  et df temp pour obetnir un df_finale
df_final_meteo=pd.concat([df_meteo,df_datetime])
df_final_meteo['DateHeure'] = pd.to_datetime(df_final_meteo['DateHeure'],utc=True)

In [ ]:
#visualisation
df_final_meteo.head()

,DateHeure,year,region,t
0,2021-01-01 00:00:00+00:00,2021,Normandie,272.775
1,2021-01-01 03:00:00+00:00,2021,Normandie,273.525
2,2021-01-01 06:00:00+00:00,2021,Normandie,272.925
3,2021-01-01 09:00:00+00:00,2021,Normandie,273.775
4,2021-01-01 12:00:00+00:00,2021,Normandie,276.55


In [ ]:
#création df final pour les données température en France de 2021 à 2022 par région 
# traitement des valeurs manquantes par interpolation linéaire 
# retrait des territoires  Mayotte et Corse
temp=df_final_meteo.copy()
temp=temp.sort_values(by=["region","DateHeure"])
temp = temp.reset_index(drop=True)
temp["t"]=temp["t"].astype(float)

#remplacement de NAN des temp manquantes par interpolation linéaire
temp['temp']=temp['t'].interpolate(method='linear') #nearest
temp=temp.iloc[:,[0,1,2,4]]
df_final_meteo=temp[~temp['region'].isin(["Mayotte","CORSE"])]  #exclure Mayotte et Corse

print(df_final_meteo.shape)
#vérification des NAN dans la colonne "temp"
df_final_meteo.isna().sum()

(420437, 4)


DateHeure    0
year         0
region       0
temp         0
dtype: int64

In [ ]:
df_final_meteo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 420437 entries, 0 to 490511
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype              
---  ------      --------------   -----              
 0   DateHeure   420437 non-null  datetime64[ns, UTC]
 1   year        420437 non-null  int64              
 2   region      420437 non-null  object             
 3   temperatue  420437 non-null  float64            
dtypes: datetime64[ns, UTC](1), float64(1), int64(1), object(1)
memory usage: 16.0+ MB


In [ ]:
#vérification des dimensions de  données de chaque région
for region in df_final_meteo["region"].unique().tolist():
      T=df_final_meteo.loc[(df_final_meteo["region"]==region)]
      print(region," : ",T.shape)

Auvergne-Rhône-Alpes  :  (35038, 4)
Bourgogne-Franche-Comté  :  (35022, 4)
Bretagne  :  (35038, 4)
Centre-Val de Loire  :  (35038, 4)
Grand-Est  :  (35038, 4)
Hauts-de-France  :  (35038, 4)
Ile-de-France  :  (35035, 4)
Normandie  :  (35038, 4)
Nouvelle-Aquitaine  :  (35038, 4)
Occitanie  :  (35038, 4)
PACA  :  (35038, 4)
Pays-de-la-Loire  :  (35038, 4)


###merge df final des régions 

In [ ]:
#merge df consommation et df météo des régions
df_electricity_sort=df_conso_region.sort_values(by=["region","DateHeure"])
df_meteo_sort=df_final_meteo.sort_values(by=["region","DateHeure"])
df_merged = pd.merge(df_electricity_sort, df_meteo_sort, on=['DateHeure','region'])
df_region=df_merged[["DateHeure","region","conso","temp"]]
#df_region['conso']=df_region['conso'].astype(int)

### ajout de la température  dans le df de consommation electrique  au niveau natianle

ajout temp moyenne nationnale pour les 30 min

In [ ]:
#fonction qui calcule la température moyenne nationale pour chaque 30mn
def agg_func(x):
    return pd.DataFrame.from_dict({
        'temp': x['temp'].mean()
    }, orient='index').transpose()

df_meteo1=df_final_meteo.copy()
df_meteo_sort=df_meteo1.sort_values(by=["DateHeure"])
#application de la fonction pou calculer la temp moyenne
temp_mean_nat= df_meteo_sort.groupby('DateHeure').apply(agg_func).reset_index()
temp_mean_nat.head()

,DateHeure,level_1,temp
0,2021-01-01 00:00:00+00:00,0,274.741944
1,2021-01-01 00:30:00+00:00,0,274.671505
2,2021-01-01 01:00:00+00:00,0,274.601065
3,2021-01-01 01:30:00+00:00,0,274.530625
4,2021-01-01 02:00:00+00:00,0,274.460185


In [ ]:
# jout de la tempéature moyenne nationale dans le df_conso_nationale
df_conso_nat_finale=pd.merge(df_conso_nat,temp_mean_nat,on=['DateHeure'])

In [ ]:
df_conso_nat_finale

,region,datetime,year,DateHeure,heure,conso,level_1,temp
0,France,2021-01-01,2021,2021-01-01 00:00:00+00:00,00:00:00,67010,0,274.741944
1,France,2021-01-01,2021,2021-01-01 00:30:00+00:00,00:30:00,67071,0,274.671505
2,France,2021-01-01,2021,2021-01-01 01:00:00+00:00,01:00:00,65052,0,274.601065
3,France,2021-01-01,2021,2021-01-01 01:30:00+00:00,01:30:00,64918,0,274.530625
4,France,2021-01-01,2021,2021-01-01 02:00:00+00:00,02:00:00,64376,0,274.460185
...,...,...,...,...,...,...,...,...
35033,France,2022-12-31,2022,2022-12-31 21:30:00+00:00,21:30:00,43971,0,284.921227
35034,France,2022-12-31,2022,2022-12-31 22:00:00+00:00,22:00:00,43506,0,283.521204
35035,France,2022-12-31,2022,2022-12-31 22:30:00+00:00,22:30:00,44566,0,282.121181
35036,France,2022-12-31,2022,2022-12-31 23:00:00+00:00,23:00:00,46232,0,280.721157


In [ ]:
# je concatene tout (df_region et df_conso_nat_finale) pour avoir un seul df final apres tous les prétraitement
df_global=pd.concat([df_region,df_conso_nat_finale])
df_global=df_global[["DateHeure","region","conso","temp"]]
df_global.shape

(454707, 4)

In [ ]:
#visualisation 
df_global

,DateHeure,region,conso,temp
0,2021-01-01 00:00:00+00:00,Auvergne-Rhône-Alpes,8576,276.125000
1,2021-01-01 00:30:00+00:00,Auvergne-Rhône-Alpes,8514,276.012500
2,2021-01-01 01:00:00+00:00,Auvergne-Rhône-Alpes,8273,275.900000
3,2021-01-01 01:30:00+00:00,Auvergne-Rhône-Alpes,8253,275.787500
4,2021-01-01 02:00:00+00:00,Auvergne-Rhône-Alpes,8234,275.675000
...,...,...,...,...
35033,2022-12-31 21:30:00+00:00,France,43971,284.921227
35034,2022-12-31 22:00:00+00:00,France,43506,283.521204
35035,2022-12-31 22:30:00+00:00,France,44566,282.121181
35036,2022-12-31 23:00:00+00:00,France,46232,280.721157


In [ ]:
df_global.isna().sum()

DateHeure    0
region       0
conso        0
temp         0
dtype: int64

In [ ]:
#sauvegarde du df final en csv dans mon drive
# enregistrement du df final de consommation électrique parr région des années 2021 et 2022
directory="/content/drive/MyDrive/MALIA/FORCASTING/projet_forecasting/data_RTE_METEO"
os.chdir(directory)
df_global.to_csv("df_prevision_electricity.csv",index=False)

### ajout des variables temps dans le df_global

In [ ]:
def df_to_matrix(df):
      df_mc=df_global.copy()
      df_mc['heure'] = df_mc['DateHeure'].dt.hour
      df_mc['hour'] = df_mc['DateHeure'].dt.strftime('%H:%M')
      df_mc['date'] = df_mc['DateHeure'].dt.strftime('%Y-%m-%d')
      df_mc['date'] = pd.to_datetime(df_mc['date'])
      #df_mc['period_journey'] = df_mc['heure'].apply(lambda x: 'Matin' if x <12 else ('Apres-midi' if x <18 else ('Soir' if x < 22  else 'Nuit')))
      df_mc['period_journey'] = df_mc['heure'].apply(lambda x: 0 if x <12 else (1 if x <18 else (2 if x < 22  else 3)))
      df_matrix = df_mc.pivot_table(index=["date"], 
                                                  columns=["hour","region"], 
                                                  values="conso")
      # Reconstruction des informations calendrier utiles

      # Créez une colonne de jours de la semaine en utilisant la fonction weekday de calendar
      df_matrix["weekday"] = [calendar.day_name[date.weekday()] 
                                            for date in df_matrix.index]

      # Ajoutez une colonne "Weekend" qui indique si la date est un week-end ou non
      df_matrix['Weekend'] =[True if date.weekday() in [calendar.SATURDAY, calendar.SUNDAY] else False   for date in df_matrix.index]
      df_matrix['month'] = df_matrix.index.strftime('%B')
      df_matrix["year"] = df_matrix.index.year
      # création d'une colonne pour indiquer si le jour est férié ou pas
      # objet holiday France
      fr_holidays = holidays.country_holidays('FR')
      df_matrix["isHolyday"]=df_matrix.index.map(lambda d: d in fr_holidays) 
      return df_matrix

In [ ]:
#appliation de la fonction
df_matix=df_to_matrix(df_global)

In [ ]:
#sauvegarde
#sauvegarde df global
df_matix.to_csv("df_ajout_var_calendar.csv",index=False)

In [ ]:
df_matix

hour                      00:00                                   \
region     Auvergne-Rhône-Alpes Bourgogne-Franche-Comté Bretagne   
date                                                               
2021-01-01               8576.0                  2671.0   3984.0   
2021-01-02               8111.0                  2352.0   3018.0   
2021-01-03               7547.0                  2334.0   3219.0   
2021-01-04               6566.0                  2081.0   2608.0   
2021-01-05               6671.0                  2175.0   2656.0   
...                         ...                     ...      ...   
2022-12-27                  NaN                     NaN      NaN   
2022-12-28               7797.0                  2027.0   2812.0   
2022-12-29               7691.0                  1814.0   2827.0   
2022-12-30               7432.0                  1975.0   2936.0   
2022-12-31               7167.0                  1695.0   2546.0   

hour                                                               \
region     Centre-Val de Loire   France Grand-Est Hauts-de-France   
date                                                                
2021-01-01              2900.0  67010.0    5321.0          6340.0   
2021-01-02              2648.0  58834.0    5383.0          6550.0   
2021-01-03              2594.0  57017.0    5035.0          5937.0   
2021-01-04              1888.0  48189.0    4611.0          5406.0   
2021-01-05              2130.0  50370.0    4781.0          5625.0   
...                        ...      ...       ...             ...   
2022-12-27                 NaN  50799.0       NaN             NaN   
2022-12-28              2356.0  52142.0    4092.0          6507.0   
2022-12-29              2221.0  51097.0    4002.0          5316.0   
2022-12-30              2315.0  51553.0    3807.0          6306.0   
2022-12-31              2099.0  48022.0    4074.0          6133.0   

hour                                                                     \
region     Ile-de-France Normandie Nouvelle-Aquitaine Occitanie    PACA   
date                                                                      
2021-01-01        9914.0    4326.0             6512.0    5840.0  6395.0   
2021-01-02        8337.0    3578.0             5083.0    5051.0  5315.0   
2021-01-03        8364.0    3625.0             5205.0    4678.0  4995.0   
2021-01-04        6577.0    2801.0             4249.0    4179.0  4343.0   
2021-01-05        7274.0    3154.0             4442.0    4176.0  4451.0   
...                  ...       ...                ...       ...     ...   
2022-12-27           NaN       NaN                NaN       NaN     NaN   
2022-12-28        8249.0    3416.0             4795.0    4540.0  4793.0   
2022-12-29        7919.0    3287.0             4685.0    4667.0  4731.0   
2022-12-30        8180.0    3480.0             4857.0    4498.0  4669.0   
2022-12-31        7483.0    3061.0             4511.0    4397.0  4781.0   

hour                                       00:30                          \
region     Pays-de-la-Loire Auvergne-Rhône-Alpes Bourgogne-Franche-Comté   
date                                                                       
2021-01-01           4230.0               8514.0                  2618.0   
2021-01-02           3408.0               7829.0                  2294.0   
2021-01-03           3485.0               7562.0                  2304.0   
2021-01-04           2881.0               6470.0                  2026.0   
2021-01-05           2834.0               6501.0                  2104.0   
...                     ...                  ...                     ...   
2022-12-27              NaN               6719.0                  2020.0   
2022-12-28           3371.0               7573.0                  2000.0   
2022-12-29           3296.0               7225.0                  1765.0   
2022-12-30           3455.0               7148.0                  1956.0   
2022-12-31           3063.0              